Il faudrait Donner la possibilité à l'utilisateur, soit de traiter tous les fichiers qui sont dans le current folder, soit de traiter spécifiquement un seul fichier.

Il faudrait aussi verifier si le fichier .json existe déjà, pour savoir si on fait une update ou si les data ont déjà été récupérées

#### Import packages

In [6]:
# Image processing
from PIL import Image 
import json

# for working on excel file
import xlrd  

# To get a list of the file in a folder
from glob import glob  

import numpy as np
import time

In [10]:
# List all .xlsm files (contain BB coordinate and orientation) in the folder 'data'
xls_list = glob("./data/*.xlsm")

# List all .tif images (where BB are annotated) in the folder 'data'
tif_list = glob("./data/*.tif")

if len(xls_list) != len(tif_list):
    raise Exception("A xls/tif file is missing: find the missing file or remove the extra file" )
    
for i in range(max(len(xls_list), len(tif_list))):
    if xls_list[i].replace('.xlsm', '') != tif_list[i].replace('.tif', ''):
        raise Exception("{} \n or \n {} is not appariate with a tif/xsl".format(xls_list[i], tif_list[i]))
        
del tif_list

In [8]:
# Size of the expected image output
# Cropped image is a square with odd number of pixels (32 pixels askes => 33 outputed) like this the centriole is exactly in the middle
size_img = 32

# Define Image size openable with PIL (default size is too small)
Image.MAX_IMAGE_PIXELS = 10000000000

# Initialisation of the centriole (~Basal body) coordinate and angle list
count_img = 0

# Do you want to save the database:
SAVE_DB = True

# Instantiate the dictionnary of unwanted centriole
centriole_trouble = {'Nan_in_image' : [], 'Black_centriole' : [], 'Angle' : [], 'Coordinates' : [], 'ID' :  []}

# Iteration trough the excel file
for xls_name in xls_list:
    
    t1 = time.time()
    
    tif_name = xls_name.replace('.xlsm', '.tif')
    file_name = xls_name.replace('.xlsm', '')
    file_name = file_name[file_name.index('\\')+1:]
    
    print("File: {} ".format(file_name), end ='')
    #print("File: {}       {}/{}     time: ".format(file_name, count_img + 1, len(xls_list), t2-t1), end ='')

    
    centriole_database = {file_name: [] }

    # Open the image
    with Image.open(tif_name) as tif_file:
    
        # Open the excel file 
        with xlrd.open_workbook(xls_name) as xls_file:
            # Go to appropriate sheet (index 0)
            xls_sheet = xls_file.sheet_by_index(0)

        # In the Excel file, the coordinate of BB starts at row 7
        # This variable will iterate a reinitialisation is required after each xls file
        count_centriole = 0

        for row in range(xls_sheet.nrows - 7):
            

            # Extract the centriole on the image
            if isinstance(xls_sheet.cell_value(row + 7, 3), (float, int)):
                X_to_crop = xls_sheet.cell_value(row + 7, 3) - size_img//2
                Y_to_crop = xls_sheet.cell_value(row + 7, 4) - size_img//2
                centriole_extracted = tif_file.crop((X_to_crop, Y_to_crop, X_to_crop + size_img, Y_to_crop + size_img))
                
                count_centriole += 1
           

            
                # Try to transform the extracted centriole into numpy array (Nan rais an error)
                try:
                    centriole_extracted = np.asarray(centriole_extracted, dtype = "uint16")

                except:
                    trouble_dictionnary['Nan_in_image'].append(coord = { "file_name" : file_name,
                                                                        "ID" : count_centriole,
                                                                         "X" : xls_sheet.cell_value(row + 7, 3),
                                                                         "Y" : xls_sheet.cell_value(row + 7, 4), 
                                                                         "angle" : xls_sheet.cell_value(row + 7, 1)})

                # Try to aggregate all information of the centriole
                try:   
                    coord = {"ID": count_centriole,
                             "X" : xls_sheet.cell_value(row + 7 , 3),
                             "Y" : xls_sheet.cell_value(row + 7 , 4), 
                             "angle" : xls_sheet.cell_value(row + 7, 1),
                             "image" : centriole_extracted.tolist()}  
                except:
                    print('trouble in the creation of centriole coordinate')


                # Check if we have value 
                if not isinstance(coord["ID"], (float, int)):
                    coord["file_name"] = file_name
                    centriole_trouble['ID'].append(coord)
                    print('angle => ID: {}'.format(coord['ID']))

                elif not isinstance(coord["X"], (float, int)) or not isinstance(coord["Y"], (float, int)):
                    coord["file_name"] = file_name
                    centriole_trouble['Coordinates'].append(coord)
                    print('angle => ID: {}, X: {} - Y: {}'.format(coord['ID'], coord['X'], coord['Y']))

                elif not isinstance(coord["angle"], (float, int)):
                    coord["file_name"] = file_name
                    centriole_trouble['Angle'].append(coord)
                    print('angle => ID: {}, Angle: {}'.format(coord['ID'], coord['angle']))

                elif centriole_extracted.max() == 0:
                    coord["file_name"] = file_name
                    centriole_trouble['Black_centriole'].append(coord)
                    print('angle => ID: {}'.format(coord['ID']))

                else:   
                    try: 
                        # Add the centriole to the dictionnary
                        centriole_database[file_name].append(coord)
                    except: 
                        print('centriole not append')
                        
        #print("     centriole treated: {} \n".format(len(centriole_database[file_name])))
        count_img += 1
    
    
    #Save the database
    if SAVE_DB:
        with open((file_name + '.json'), 'w') as output: ## Verifier que les fichiers tif se finissent bien en '.tif'
            json.dump(centriole_database, output)
        
    t2 = time.time()
    print("     {} centrioles    {}/{}     time: {:.1f}s".format(len(centriole_database[file_name]), count_img, len(xls_list), t2-t1))
    
        
# Save the list of problematic centriole
with open('centriole_trouble_list.json', 'w') as output:
    json.dump(centriole_trouble, output)
    
print('Extraction over')


Extraction over


In [9]:
xls_list

[]